In [1]:
import pandas as pd
import tensorflow as tf

In [2]:
data = pd.read_csv('../data/full_data.csv')
data.head()

,movieId,userId,title,genres,rating,tag
0,1,2483,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,4.0,lots of heart
1,1,31107,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.0,animation
2,1,31107,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.0,family
3,1,31107,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.0,sci-fi
4,1,82148,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,4.0,Animation


In [ ]:
genre_vocab = set('|'.join(data['genres']).split('|'))
tag_vocab = set(data['tag'].unique())

genre_column = tf.feature_column.categorical_column_with_vocabulary_list(
    'genres', list(genre_vocab))
tag_column = tf.feature_column.categorical_column_with_vocabulary_list(
    'tag', list(tag_vocab))

genre_one_hot = tf.feature_column.indicator_column(genre_column)
tag_one_hot = tf.feature_column.indicator_column(tag_column)

feature_columns = [genre_one_hot, tag_one_hot]
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)